In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
# from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
# from keras.utils import np_utils
# from skimage.transform import resize   # for resizing images
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import random as rnd
from glob import glob
from tqdm import tqdm

datadir = "/home/ephy/Projects/tda_convolve_video/data/"

from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
device='cuda:0'

from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.integrate import tplquad
from scipy.ndimage import rotate as scipy_rotate
from scipy.ndimage import shift as scipy_shift
from joblib import Parallel, delayed

In [2]:
f = open(datadir+"original/ucfTrainTestlist/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]

# open the .txt file which have names of test videos
f = open(datadir+"original/ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]

In [3]:
# creating tags
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

allVids = train.append(test, ignore_index=True)

In [6]:
# storing the frames from training videos
for i in tqdm(range(allVids.shape[0])):
    count = 0
    videoFile = allVids['video_name'][i]
    cap = cv2.VideoCapture(datadir + 'original/UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = 1 #frame sample rate
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
#             filename =datadir+'altered/allUCF101_FFR/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count
            count+=1
#             cv2.imwrite(filename, frame)
    allVids.loc[:,'frames'].iloc[i] = count
    cap.release()

  0%|          | 0/13320 [00:00<?, ?it/s]


KeyError: 'frames'

In [42]:
print(np.mean(allVids['frames']))
print(np.std(allVids['frames']))
print(np.min(allVids['frames']))
print(np.max(allVids['frames']))

186.43355855855856
97.6545364970198
29.0
1776.0


In [62]:
i=1
np.array([0,allVids['frames'][i]/2,allVids['frames'][i]-7],dtype=np.int)

array([  0,  58, 110])

In [102]:
#Factor of 2.77 shrink getws us very close to 100^2 pixels, same as experiments before, but with aspect preserved
imageSize=np.array([np.round(320/2.77),np.round(240/2.77)],dtype=np.int)

In [5]:
# storing the frames from training videos
all_image = []
for i in tqdm(range(allVids.shape[0])):
    count = 0
    videoFile = allVids['video_name'][i]
    cap = cv2.VideoCapture(datadir + 'original/UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = 1 #frame sample rate
    startPoints = np.array([0,allVids['frames'][i]/2,allVids['frames'][i]-7],dtype=np.int)
    frames = []
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            frames.append(frame)
            count+=1
    cap.release()
    frames = np.stack(frames)
    startPoints = np.array([0,(frames.shape[0]/2)-4,frames.shape[0]-8],dtype=np.int)
    clips = []
    for start in startPoints:
        clip = []
        for f in range(start,start+7):
            image = frames[f]
            image = Image.fromarray(image , 'RGB')
            image = image.resize(imageSize)
            image = np.array(image)/255
            image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])
            clip.append(image)
        clips.append(clip)
    all_image.append(np.array(clips))
all_image = np.stack(all_image)    

  0%|          | 0/13320 [00:00<?, ?it/s]


KeyError: 'frames'

In [2]:
all_images = Parallel(n_jobs=n_jobs)(delayed(one_run)(i,allvids,datadir) for i in tqdm(range(allVids.shape[0])))

NameError: name 'Parallel' is not defined

In [139]:
# filters = np.load('/home/ephy/Projects/tda_convolve_video/src/python3/VideoFeatures_355.npy', allow_pickle=True)
# With rotations
filters = np.load('/home/ephy/Projects/tda_convolve_video/src/python3/VideoFeatures_355_movandrot.npy', allow_pickle=True)
flat_filters = filters.reshape(-1, *filters.shape[-3:])

In [169]:
allVids = pd.read_csv(datadir + 'altered/all_data_guide_FFR.csv')
allVids = allVids.sort_values(by=['clip', 'frame'])
allVids_dedup = allVids.drop_duplicates(subset=['clip'], keep='first').copy()

In [170]:
# Make an interger based class to make tracking tests more simple
classXwalk = pd.DataFrame({'class': np.unique(allVids_dedup['class'])})
classXwalk['ind'] = np.array(list(range(0,101)))
allVids_dedup['class_int'] = [int(classXwalk['ind'][classXwalk['class']==c]) for c in allVids_dedup['class']]

In [167]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 180, (3,5,5))
        self.conv2 = nn.Conv3d(180, 180, (3,5,5))
        self.conv3 = nn.Conv3d(180, 180, (3,5,5))
        self.conv4 = nn.Conv3d(180, 36, (1,5,5))
        
        x= torch.randn(1,1,*all_image.shape[2:])
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 750)
        self.fc2 = nn.Linear(750, 200)
        self.fc3 = nn.Linear(200, 101)
        
        self.batch1 = nn.BatchNorm1d(self._to_linear)
        self.batch2 = nn.BatchNorm1d(750)
        self.batch3 = nn.BatchNorm1d(200)
#         self.drop1 = nn.Dropout(p=0.2)
        self.drop2 = nn.Dropout(p=0.2)
        
        
    def convs(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]*x[0].shape[3]
            
        return x
    
    def forward(self, x):
        # reshape inside of array
        x = self.convs(x)
#         x = self.drop1(x)
        x = x.view(-1, self._to_linear)
#         x = self.batch1(x)
        x = F.leaky_relu(self.fc1(x))
#         x = self.batch2(x)
        x = F.leaky_relu(self.fc2(x))
        x = self.batch3(x)
        x = self.drop2(x)
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [168]:
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-5)
torch.cuda.memory_allocated()*1e-9

0.7924976640000001

In [199]:
def test(net, X, y, batchSize):
    confusionMatrix = np.zeros([101,101], dtype=np.int8)
    testingSeq = list(range(0,y.shape[0]+1,batchSize))
    testingSeq.append(y.shape[0]+1)
    testingSeq = np.array(testingSeq)
    with torch.no_grad():
        for i in range(len(testingSeq)-1):
            data = X[testingSeq[i]:testingSeq[i+1]].view(-1,1,*all_image.shape[2:]).to(device)
            out = torch.argmax(net(data),axis=1).cpu()
            yt = torch.argmax(y[testingSeq[i]:testingSeq[i+1]],axis=1)
            for pred,label in zip(out,yt):
                confusionMatrix[label][pred] +=1
        del data
        del out        
        return confusionMatrix

In [ ]:
EPOCHS = 40
# # Batch size of 26+1, because normalization requires same batch size and 4472%26=0 (very close to even split) (same logic for training)
testingBatchSize=20
trainingBatchSize=20
results = [[],[],[]]
lossrec = [[],[],[]]

# rnd.seed(1)
classMat = np.eye(101,dtype=np.float)
# for foldIndex in range(3):
foldIndex=0
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-5)
# Set filters in 2 layers
with torch.no_grad():
    for i,weights in enumerate(flat_filters):
        net.conv1.weight[i][0] = torch.nn.Parameter(torch.tensor(weights)).to(device)
#             for g,filts in enumerate(net.conv2.weight):
#                 net.conv2.weight[i][g] = torch.nn.Parameter(torch.tensor(flat_filters[g])).to(device)
    net.conv1.requires_grad=False
#         net.conv2.requires_grad=False
print('Network Reset.')


X_train, X_test, y_train, y_test = train_test_split(all_image, allVids_dedup['class_int'],
                                                    shuffle=True, random_state=foldIndex,
                                                    test_size=0.33, stratify = allVids_dedup['class_int'])

# # Flatten out clips in training to triple sample size
X_train = np.concatenate([X_train[:,0,...],X_train[:,1,...],X_train[:,2,...]])
y_train = y_train.tolist()*3

# Just use one clip in testing
X_test = X_test[:,0,...]

y_train = torch.tensor([classMat[c] for c in y_train],dtype=torch.float32)
y_test = torch.tensor([classMat[c] for c in y_test],dtype=torch.float32)
X_train = torch.tensor(X_train,dtype=torch.float32)
X_test =  torch.tensor(X_test,dtype=torch.float32)
y_train = torch.tensor(np.array(y_train),dtype=torch.float32)
y_test = torch.tensor(np.array(y_test),dtype=torch.float32)

trainingSeq = list(range(0,y_train.shape[0],trainingBatchSize))
trainingSeq.append(y_train.shape[0]+1)
trainingSeq = np.array(trainingSeq)

# Grab 5 tests in final epoch (4 + last one run after training loop)
finalEpochTests = list(range(int((len(trainingSeq)-1)/5),len(trainingSeq)-1,int((len(trainingSeq)-1)/5)))

for epoch in range(EPOCHS):
    for i in range(len(trainingSeq)-1):
        X=X_train[trainingSeq[i]:trainingSeq[i+1]].view(-1,1,*all_image.shape[2:])
        X=X.to(device)
        y=y_train[trainingSeq[i]:trainingSeq[i+1]]
        y=y.to(device)
        net.zero_grad()
        output = net(X)
        loss = nn.CrossEntropyLoss()(output, torch.argmax(y, axis=1))
        lossrec[foldIndex].append(float(loss.tolist()))
        loss.backward()
        optimizer.step()
        del loss
        del output
        del X
        del y
        if epoch==(EPOCHS-1) and (i in finalEpochTests):
            print(foldIndex,'Final Epoch',i)
            results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
    print('Fold: ', foldIndex, 'Epoch: ',epoch)
    results[foldIndex].append(test(net, X_test, y_test, testingBatchSize))
    cm = results[foldIndex][len(results[foldIndex])-1]
    print('Accuracy: ' , np.sum(np.diag(cm))/np.sum(cm))

np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/losses_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',lossrec)
np.save('/home/ephy/Projects/tda_convolve_video/data/models/results/confuseMats_1kflayer355_4layer_fold'+str(foldIndex)+'.npy',results)    

Network Reset.
Fold:  0 Epoch:  0
Accuracy:  0.1897179253867152
Fold:  0 Epoch:  1
Accuracy:  0.34121929026387626
Fold:  0 Epoch:  2
Accuracy:  0.42197452229299365
Fold:  0 Epoch:  3
Accuracy:  0.4654231119199272
Fold:  0 Epoch:  4
Accuracy:  0.5015923566878981
Fold:  0 Epoch:  5
Accuracy:  0.525705186533212
